In [100]:
import torch
import torch.nn as nn
import pandas as pd
import yaml
import logging
import sys
import torch.nn.functional as F
import numpy as np

from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import argparse
import visdom

from __future__ import print_function
import os
from collections import defaultdict
import numpy as np
import scipy.stats
import torch
from torch.distributions import constraints
from pyro import poutine
from pyro.contrib.autoguide import AutoDelta
from pyro.optim import Adam
from pyro.infer import SVI, TraceEnum_ELBO, config_enumerate, infer_discrete
from matplotlib import pyplot
import pyro
import pyro.distributions as dist
from sklearn import preprocessing
import yaml
# """Device"""
# if torch.cuda.is_available():
#     dev = "cuda:0"
# else:
#     dev = "cpu"
# device = torch.device(dev)

# """Load configuration"""
# config_path = "/home/trduong/Data/counterfactual_fairness_game_theoric/configuration.yml"
# conf = load_config(config_path)


# log_file = conf['log_train_compas']
# data_path = conf['data_compas']
# ae_path = conf['compas_encoder']

In [101]:
"""Load configuration"""
with open("/home/trduong/Data/counterfactual_fairness_game_theoric/configuration.yml", 'r') as stream:
    try:
        conf = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [2]:
K = 2 
weights = pyro.sample('weights', dist.Dirichlet(0.5 * torch.ones(K)))

In [3]:
weights

tensor([0.9968, 0.0032])

In [4]:
0.5 * torch.ones(K)

tensor([0.5000, 0.5000])

In [5]:
"""Device"""
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)

In [6]:
from torch.distributions.poisson import Poisson
m = Poisson(torch.tensor([4]).to(device))


In [7]:
import pyro.distributions as dist


In [8]:
test = dist.Poisson(rate = 0.1)

In [9]:
arr = []
for i in range(10000):
    arr.append(pyro.sample("Race", test).numpy().tolist())

In [10]:
arr

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0

In [11]:
from collections import Counter
Counter(arr)

Counter({0.0: 9072, 1.0: 891, 2.0: 36, 3.0: 1})

# Compas

In [102]:
df = pd.read_csv("https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv")

In [103]:
df.columns

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')

In [104]:
df.describe()

,id,age,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,is_recid,r_days_from_arrest,violent_recid,is_violent_recid,decile_score.1,v_decile_score,priors_count.1,start,end,event,two_year_recid
count,7214.000000,7214.000000,7214.000000,7214.000000,7214.000000,7214.000000,7214.000000,6907.000000,7192.000000,7214.000000,2316.000000,0.0,7214.000000,7214.000000,7214.000000,7214.000000,7214.000000,7214.000000,7214.000000,7214.000000
mean,5501.255753,34.817993,0.067230,4.509565,0.090934,0.109371,3.472415,3.304763,57.731368,0.481148,20.269430,NaN,0.113529,4.509565,3.691849,3.472415,11.465068,553.436651,0.382867,0.450652
std,3175.706870,11.888922,0.473972,2.856396,0.485239,0.501586,4.882538,75.809505,329.740215,0.499679,74.871668,NaN,0.317261,2.856396,2.510148,4.882538,46.954563,399.020583,0.486120,0.497593
min,1.000000,18.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-414.000000,0.000000,0.000000,-1.000000,NaN,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2735.250000,25.000000,0.000000,2.000000,0.000000,0.000000,0.000000,-1.000000,1.000000,0.000000,0.000000,NaN,0.000000,2.000000,1.000000,0.000000,0.000000,148.250000,0.000000,0.000000
50%,5509.500000,31.000000,0.000000,4.000000,0.000000,0.000000,2.000000,-1.000000,1.000000,0.000000,0.000000,NaN,0.000000,4.000000,3.000000,2.000000,0.000000,530.500000,0.000000,0.000000
75%,8246.500000,42.000000,0.000000,7.000000,0.000000,0.000000,5.000000,0.000000,2.000000,1.000000,1.000000,NaN,0.000000,7.000000,5.000000,5.000000,1.000000,914.000000,1.000000,1.000000
max,11001.000000,96.000000,20.000000,10.000000,13.000000,17.000000,38.000000,1057.000000,9485.000000,1.000000,993.000000,NaN,1.000000,10.000000,10.000000,38.000000,937.000000,1186.000000,1.000000,1.000000


In [105]:
df.c_charge_degree.value_counts(), df.c_charge_desc.value_counts(), df.type_of_assessment.value_counts()

(F    4666
 M    2548
 Name: c_charge_degree, dtype: int64,
 Battery                           1156
 arrest case no charge             1137
 Possession of Cocaine              474
 Grand Theft in the 3rd Degree      425
 Driving While License Revoked      200
                                   ... 
 Sex Battery Deft 18+/Vict 11-        1
 Purchase/P/W/Int Cannabis            1
 Possession Of Buprenorphine          1
 Criminal Attempt 3rd Deg Felon       1
 Cruelty to Animals                   1
 Name: c_charge_desc, Length: 437, dtype: int64,
 Risk of Recidivism    7214
 Name: type_of_assessment, dtype: int64)

In [106]:
df.c_charge_desc.value_counts().reset_index()[:20]

,index,c_charge_desc
0,Battery,1156
1,arrest case no charge,1137
2,Possession of Cocaine,474
3,Grand Theft in the 3rd Degree,425
4,Driving While License Revoked,200
5,Driving Under The Influence,135
6,Felony Battery (Dom Strang),100
7,Grand Theft (Motor Vehicle),98
8,Pos Cannabis W/Intent Sel/Del,97
9,Felony Driving While Lic Suspd,96


In [107]:
categorical_features=['age_cat', 'c_charge_degree'],

features_to_keep=['sex',
                  'age', 
                  'age_cat',
                  'race',
                  'juv_fel_count', 
                  'juv_misd_count',
                  'juv_other_count',
                  'priors_count',
                  'c_charge_degree', 
                  'two_year_recid']
        

In [108]:
conf['data_compas']

'/home/trduong/Data/counterfactual_fairness_game_theoric/data/processed_compas.csv'

In [112]:
df = df[features_to_keep]

In [121]:
df

,sex,age,age_cat,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_degree,two_year_recid
0,Male,69,Greater than 45,Non Caucasian,0,0,0,0,F,0
1,Male,34,25 - 45,Non Caucasian,0,0,0,0,F,1
2,Male,24,Less than 25,Non Caucasian,0,0,1,4,F,1
3,Male,23,Less than 25,Non Caucasian,0,1,0,1,F,0
4,Male,43,25 - 45,Non Caucasian,0,0,0,2,F,0
...,...,...,...,...,...,...,...,...,...,...
7209,Male,23,Less than 25,Non Caucasian,0,0,0,0,F,0
7210,Male,23,Less than 25,Non Caucasian,0,0,0,0,F,0
7211,Male,57,Greater than 45,Non Caucasian,0,0,0,0,F,0
7212,Female,33,25 - 45,Non Caucasian,0,0,0,3,M,0


In [123]:
df.age_cat.value_counts()

25 - 45            4109
Greater than 45    1576
Less than 25       1529
Name: age_cat, dtype: int64

In [122]:
df.c_charge_degree.value_counts()

F    4666
M    2548
Name: c_charge_degree, dtype: int64

In [115]:
df['race'] = ['Caucasian' if x == 'Caucasian' else "Non Caucasian" for x in df['race']]

In [120]:
df

,sex,age,age_cat,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_degree,two_year_recid
0,Male,69,Greater than 45,Non Caucasian,0,0,0,0,F,0
1,Male,34,25 - 45,Non Caucasian,0,0,0,0,F,1
2,Male,24,Less than 25,Non Caucasian,0,0,1,4,F,1
3,Male,23,Less than 25,Non Caucasian,0,1,0,1,F,0
4,Male,43,25 - 45,Non Caucasian,0,0,0,2,F,0
...,...,...,...,...,...,...,...,...,...,...
7209,Male,23,Less than 25,Non Caucasian,0,0,0,0,F,0
7210,Male,23,Less than 25,Non Caucasian,0,0,0,0,F,0
7211,Male,57,Greater than 45,Non Caucasian,0,0,0,0,F,0
7212,Female,33,25 - 45,Non Caucasian,0,0,0,3,M,0


In [119]:
df.to_csv(conf['data_compas'], index = False)

In [117]:
df.two_year_recid.value_counts()

0    3963
1    3251
Name: two_year_recid, dtype: int64

In [111]:
# le = preprocessing.LabelEncoder()
# df[c] = le.fit_transform(df[c])
# le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
# print(c)
# print(le_name_mapping)

# Adult

In [16]:
path = "/data/trduong/counterfactual_fairness_game_theoric/data/processed_adult.csv"
df = pd.read_csv(path)

In [17]:
df

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,39,Government,Bachelors,Single,White-Collar,White,Male,40,0
1,50,Self-Employed,Bachelors,Married,White-Collar,White,Male,13,0
2,38,Private,HS-grad,Divorced,Blue-Collar,White,Male,40,0
3,53,Private,School,Married,Blue-Collar,Other,Male,40,0
4,28,Private,Bachelors,Married,Professional,Other,Female,40,0
...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc,Married,Service,White,Female,38,0
32557,40,Private,HS-grad,Married,Blue-Collar,White,Male,40,1
32558,58,Private,HS-grad,Widowed,White-Collar,White,Female,40,0
32559,22,Private,HS-grad,Single,White-Collar,White,Male,20,0


In [18]:
df = pd.read_csv("/data/trduong/counterfactual_fairness_game_theoric/data/bank.csv")

In [19]:
df.deposit.value_counts()

no     5873
yes    5289
Name: deposit, dtype: int64

In [20]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'deposit'],
      dtype='object')

In [21]:
df.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
deposit      object
dtype: object

In [22]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,unknown,no
11158,39,services,married,secondary,no,733,no,no,unknown,16,jun,83,4,-1,0,unknown,no
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,unknown,no
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,no


In [23]:
df.default.value_counts()

no     10994
yes      168
Name: default, dtype: int64

# Bank

In [24]:
bank = pd.read_csv("/home/trduong/Data/counterfactual_fairness_game_theoric/data/bank.csv")

In [25]:
bank

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,unknown,no
11158,39,services,married,secondary,no,733,no,no,unknown,16,jun,83,4,-1,0,unknown,no
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,unknown,no
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,no


In [26]:
bank.campaign.value_counts()

1     4798
2     3028
3     1321
4      771
5      378
6      265
7      139
8      128
9       72
10      52
11      40
13      30
12      29
14      15
17      14
15      13
16       9
18       8
21       8
24       5
19       5
20       5
30       4
22       4
26       3
23       3
25       3
43       2
29       2
32       2
63       1
33       1
41       1
31       1
28       1
27       1
Name: campaign, dtype: int64

In [27]:
continuous = ['age', 'balance', 'duration', 'previous']
categorical = ['job', 'education', 'housing', 'loan', 'poutcome']
sensitive = ['marital']
target = 'deposit'



In [28]:
## define custom tag handler
def join(loader, node):
    seq = loader.construct_sequence(node)
    return ''.join([str(i) for i in seq])

## register the tag handler
yaml.add_constructor('!join', join)

In [29]:
with open("/home/trduong/Data/counterfactual_fairness_game_theoric/configuration.yml", 'r') as stream:
    try:
        conf = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [30]:
conf = yaml.load("/home/trduong/Data/counterfactual_fairness_game_theoric/configuration.yml", Loader=yaml.Loader)

In [31]:
df = pd.read_csv("/home/trduong/Data/counterfactual_fairness_game_theoric/data/bank.csv")

In [32]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,unknown,no
11158,39,services,married,secondary,no,733,no,no,unknown,16,jun,83,4,-1,0,unknown,no
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,unknown,no
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,no


In [33]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


In [34]:
df.deposit.value_counts()

no     5873
yes    5289
Name: deposit, dtype: int64

In [35]:
df['deposit'] = df['deposit'].eq('yes').mul(1)

In [36]:
df.deposit.value_counts()

0    5873
1    5289
Name: deposit, dtype: int64

In [37]:
df.to_csv("/home/trduong/Data/counterfactual_fairness_game_theoric/data/processed_bank.csv")

In [38]:
df = pd.read_csv("/data/trduong/counterfactual_fairness_game_theoric/data/compas.csv")

In [39]:
df

,c_charge_degree,age_cat,race,gender,score_text,priors_count,two_year_recid
0,F,Greater than 45,Other,Male,Low,0,0
1,F,25 - 45,African-American,Male,Low,0,1
2,F,Less than 25,African-American,Male,Low,4,1
3,M,25 - 45,Other,Male,Low,0,0
4,F,25 - 45,Caucasian,Male,Medium,14,1
...,...,...,...,...,...,...,...
6167,F,Less than 25,African-American,Male,Medium,0,0
6168,F,Less than 25,African-American,Male,Low,0,0
6169,F,Greater than 45,Other,Male,Low,0,0
6170,M,25 - 45,African-American,Female,Low,3,0


In [40]:
df.race.value_counts()

African-American    3175
Caucasian           2103
Hispanic             509
Other                343
Asian                 31
Native American       11
Name: race, dtype: int64

In [41]:
def mapping_race(x):
    if x != 'Caucasian':
        return "Non caucasian"
    else:
        return x
    

In [42]:
df['race'] = df['race'].map(mapping_race)

In [43]:
df.race.value_counts()

Non caucasian    4069
Caucasian        2103
Name: race, dtype: int64

In [44]:
df.to_csv("/data/trduong/counterfactual_fairness_game_theoric/data/processed_compas.csv", index = False)

In [56]:
test = dist.Multinomial(total_count=7, probs=torch.tensor([0.1, 0.2, 0.4]))

In [57]:
arr = []
for i in range(10000):
    arr.append(pyro.sample("Race", test).numpy().tolist())

In [58]:
arr

[[0.0, 5.0, 2.0],
 [0.0, 4.0, 3.0],
 [0.0, 5.0, 2.0],
 [3.0, 2.0, 2.0],
 [1.0, 1.0, 5.0],
 [2.0, 0.0, 5.0],
 [0.0, 1.0, 6.0],
 [1.0, 5.0, 1.0],
 [0.0, 2.0, 5.0],
 [1.0, 0.0, 6.0],
 [0.0, 4.0, 3.0],
 [0.0, 1.0, 6.0],
 [1.0, 1.0, 5.0],
 [0.0, 1.0, 6.0],
 [1.0, 1.0, 5.0],
 [1.0, 1.0, 5.0],
 [0.0, 4.0, 3.0],
 [1.0, 2.0, 4.0],
 [0.0, 0.0, 7.0],
 [1.0, 3.0, 3.0],
 [0.0, 3.0, 4.0],
 [2.0, 1.0, 4.0],
 [2.0, 1.0, 4.0],
 [0.0, 2.0, 5.0],
 [0.0, 2.0, 5.0],
 [1.0, 1.0, 5.0],
 [0.0, 2.0, 5.0],
 [0.0, 2.0, 5.0],
 [0.0, 1.0, 6.0],
 [1.0, 2.0, 4.0],
 [1.0, 4.0, 2.0],
 [1.0, 3.0, 3.0],
 [0.0, 3.0, 4.0],
 [1.0, 0.0, 6.0],
 [0.0, 1.0, 6.0],
 [0.0, 2.0, 5.0],
 [1.0, 0.0, 6.0],
 [2.0, 4.0, 1.0],
 [0.0, 3.0, 4.0],
 [1.0, 4.0, 2.0],
 [1.0, 4.0, 2.0],
 [0.0, 2.0, 5.0],
 [0.0, 2.0, 5.0],
 [2.0, 2.0, 3.0],
 [1.0, 1.0, 5.0],
 [1.0, 2.0, 4.0],
 [2.0, 1.0, 4.0],
 [0.0, 1.0, 6.0],
 [0.0, 1.0, 6.0],
 [1.0, 1.0, 5.0],
 [1.0, 2.0, 4.0],
 [2.0, 2.0, 3.0],
 [1.0, 3.0, 3.0],
 [2.0, 1.0, 4.0],
 [1.0, 6.0, 0.0],
 [0.0, 0.0

In [67]:

exo_dist = {
    'Nrace': dist.Bernoulli(torch.tensor(0.75)),
    'Nsex': dist.Bernoulli(torch.tensor(0.5)),
    'Nknowledge': dist.Normal(R+S, torch.tensor(1.))
}


# R+S+1 - (R+S-1) 

In [68]:
arr = []
for i in range(10000):
    R = pyro.sample("Race", exo_dist['Nrace'])
    S = pyro.sample("Sex", exo_dist['Nsex'])
    K = pyro.sample("Knowledge", exo_dist['Nknowledge'])

    arr.append([R,S,K])

In [69]:
arr

[[tensor(1.), tensor(1.), tensor(0.8877)],
 [tensor(1.), tensor(1.), tensor(1.3608)],
 [tensor(1.), tensor(0.), tensor(0.9829)],
 [tensor(0.), tensor(1.), tensor(-0.0522)],
 [tensor(1.), tensor(0.), tensor(0.5704)],
 [tensor(0.), tensor(0.), tensor(2.0025)],
 [tensor(1.), tensor(0.), tensor(1.3769)],
 [tensor(1.), tensor(0.), tensor(1.7644)],
 [tensor(0.), tensor(1.), tensor(-1.1187)],
 [tensor(1.), tensor(1.), tensor(1.5448)],
 [tensor(1.), tensor(0.), tensor(1.8138)],
 [tensor(1.), tensor(1.), tensor(2.0090)],
 [tensor(1.), tensor(0.), tensor(1.3710)],
 [tensor(0.), tensor(0.), tensor(-0.6369)],
 [tensor(1.), tensor(1.), tensor(0.7765)],
 [tensor(0.), tensor(1.), tensor(0.6395)],
 [tensor(0.), tensor(1.), tensor(1.7961)],
 [tensor(1.), tensor(0.), tensor(1.6480)],
 [tensor(0.), tensor(0.), tensor(0.2705)],
 [tensor(1.), tensor(1.), tensor(0.8711)],
 [tensor(1.), tensor(1.), tensor(1.8638)],
 [tensor(1.), tensor(0.), tensor(-0.4734)],
 [tensor(1.), tensor(0.), tensor(1.9128)],
 [tenso

In [72]:
arr = []
for i in range(1000000):
    K = pyro.sample("Knowledge", dist.Normal(0, torch.tensor(1.)))
    arr.append(K)

In [73]:
min(arr), max(arr)

(tensor(-5.2779), tensor(4.6543))

In [75]:
arr

[tensor(1.1101),
 tensor(0.0900),
 tensor(-1.4873),
 tensor(-0.8048),
 tensor(-1.9798),
 tensor(0.0386),
 tensor(1.9989),
 tensor(0.4317),
 tensor(0.0900),
 tensor(-0.1840),
 tensor(0.2059),
 tensor(-0.2013),
 tensor(0.9657),
 tensor(1.3471),
 tensor(-0.1091),
 tensor(-0.4153),
 tensor(0.5961),
 tensor(-0.4845),
 tensor(0.3179),
 tensor(-1.5926),
 tensor(-0.7572),
 tensor(2.2190),
 tensor(-0.3110),
 tensor(-0.7341),
 tensor(2.0392),
 tensor(1.2016),
 tensor(1.6342),
 tensor(1.9394),
 tensor(-1.3704),
 tensor(-0.6587),
 tensor(0.6301),
 tensor(-0.0363),
 tensor(-0.4181),
 tensor(0.0698),
 tensor(-1.4080),
 tensor(0.5847),
 tensor(1.5524),
 tensor(1.3811),
 tensor(0.2271),
 tensor(-1.4035),
 tensor(1.0032),
 tensor(0.1168),
 tensor(1.8882),
 tensor(0.9227),
 tensor(1.4456),
 tensor(-0.5991),
 tensor(-0.6052),
 tensor(0.2340),
 tensor(-0.8798),
 tensor(0.7050),
 tensor(-0.1788),
 tensor(0.3994),
 tensor(-0.2052),
 tensor(0.9159),
 tensor(-0.2823),
 tensor(1.0150),
 tensor(1.1792),
 tensor